# Machine Translation(English - Hindi)

This translation model transforms the english sentences to hindi.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from pickle import load
from pickle import dump

In [ ]:
import h5py
import math
import nltk
import re 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import io

In [ ]:
# English Training Data

english_data = 'parallel\IITB.en-hi.en'
# Read in the data.
eng_lines = io.open(english_data, encoding = "utf-8").read().split('\n')

# Hindi Training Data

hindi_data = 'parallel\IITB.en-hi.hi'
# Read in the data.
hin_lines = io.open(hindi_data, encoding = "utf-8").read().split('\n')

In [ ]:
print(eng_lines[16])

In [ ]:
print(hin_lines[16])

In [ ]:
eng_lines = pd.DataFrame(eng_lines)

In [ ]:
eng_lines.head()

In [ ]:
eng_lines.columns = ['English Sentence']

In [ ]:
eng_lines.head()

In [ ]:
hin_lines = pd.DataFrame(hin_lines)

In [ ]:
hin_lines.head()

In [ ]:
hin_lines.columns = ['Hindi Sentence']

In [ ]:
hin_lines.head()

In [ ]:
eng_lines['len_english'] = eng_lines['English Sentence'].apply(lambda x:len(x.split()))

In [ ]:
hin_lines['len_hindi'] = hin_lines['Hindi Sentence'].apply(lambda x:len(x.split()))

In [ ]:
eng_lines.head()

In [ ]:
hin_lines.head()

In [ ]:
eng_lines.describe()

In [ ]:
hin_lines.describe()

In [ ]:
import re

In [ ]:
eng_lines['Removed Spaces'] = eng_lines['English Sentence'].apply(lambda x:x.replace(" ",""))

In [ ]:
eng_lines['Non Chars'] = eng_lines['Removed Spaces'].apply(lambda x:x.isalpha())

In [ ]:
eng_lines['Non Chars'][1561797]

In [ ]:
eng_lines.head()

In [ ]:
eng_lines = eng_lines[eng_lines['Non Chars']==True]

In [ ]:
len(eng_lines)

In [ ]:
eng_lines

In [ ]:
eng_lines.drop(['Removed Spaces','Non Chars'],axis=1,inplace = True)
eng_lines.head(25)

In [ ]:
eng_lines = eng_lines[eng_lines.len_english!=0]

In [ ]:
len(eng_lines)

In [ ]:
hin_lines.head()

In [ ]:
def non_hindi(sentence):
    return bool(re.search('[a-zA-Z0-9-,:"+./।<?>}{!@#$%^&*(]', sentence))

In [ ]:
non_hindi('हाइलाइट किया गया भराई का रंग और पारदर्शिता।')

In [ ]:
hin_lines['Non Hindi Chars'] = hin_lines['Hindi Sentence'].apply(non_hindi)

In [ ]:
hin_lines.head(25)

In [ ]:
hin_lines = hin_lines[hin_lines['Non Hindi Chars']==False]

In [ ]:
hin_lines.head(25)

In [ ]:
hin_lines.drop(['Non Hindi Chars'],axis = 1,inplace=True)

In [ ]:
hin_lines = hin_lines[hin_lines.len_hindi!=0]

In [ ]:
len(hin_lines)

In [ ]:
hin_lines = pd.merge(hin_lines, eng_lines, left_index=True, right_index=True)

In [ ]:
hin_lines.head(25)

In [ ]:
len(hin_lines)

In [ ]:
hin_lines.drop(['English Sentence','len_english'],axis=1,inplace=True)

In [ ]:
hin_lines.head()

In [ ]:
eng_lines.head(25)

In [ ]:
eng_lines = pd.merge(hin_lines,eng_lines,left_index=True,right_index=True)

In [ ]:
eng_lines.head()

In [ ]:
eng_lines.drop(['Hindi Sentence','len_hindi'],axis=1,inplace=True)

In [ ]:
eng_lines.head()

In [ ]:
len(eng_lines)

In [ ]:
len(hin_lines)

In [ ]:
import collections 

In [ ]:
hindi_sentences = hin_lines['Hindi Sentence']

In [ ]:
hindi_sentences.dropna(inplace=True)
hindi_sentences.reset_index(drop=True, inplace=True)

In [ ]:
hindi_sentences

In [ ]:
y = set()

In [ ]:
for i in range(0,len(hindi_sentences)):
    b = hindi_sentences[i].split()
    for i in range(0,len(b)):
        y.add(b[i])

In [ ]:
print(len(y))

In [ ]:
y

In [ ]:
corpus = []

In [ ]:
english_sentences = eng_lines['English Sentence']

In [ ]:
english_sentences.dropna(inplace=True)
english_sentences.reset_index(drop=True, inplace=True)

In [ ]:
english_sentences

In [ ]:
for i in range(0,len(english_sentences)):
    sen = re.sub('[^a-zA-Z]', ' ', english_sentences[i])
    sen = sen.lower()
    sen = sen.split()
    sen = ' '.join(sen)
    corpus.append(sen)

In [ ]:
len(corpus)

In [ ]:
corpus

In [ ]:
x = set()

In [ ]:
for i in range(0,len(corpus)):
    b = corpus[i].split()
    for i in range(0,len(b)):
        x.add(b[i])

In [ ]:
print(len(x))

In [ ]:
x

In [ ]:
english_words_counters = collections.Counter([word for sentence in corpus for word in sentence.split()])

In [ ]:
print('{} English words.'.format(len([word for sentence in corpus for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counters)))
print('30 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counters.most_common(30)))[0]) + '"')
print()

In [ ]:
hindi_words_counters = collections.Counter([word for sentence in hindi_sentences for word in sentence.split()])

In [ ]:
print('{} Hindi words.'.format(len([word for sentence in hindi_sentences for word in sentence.split()])))
print('{} unique Hindi words.'.format(len(hindi_words_counters)))
print('30 Most common words in the Hindi dataset:')
print('"' + '" "'.join(list(zip(*hindi_words_counters.most_common(30)))[0]) + '"')
print()

In [ ]:
def load_pickle_file(filename):
    return load(open(filename, 'rb'))

In [ ]:
def save_pickle_file(sentences,filename):
    dump(sentences, open(filename,'wb'))
    print('Saved: %s',filename)

In [ ]:
hindi_vocab = 'hindi_vocab.pkl'
save_pickle_file(y, hindi_vocab)

In [ ]:
english_vocab = 'english_vocab.pkl'
save_pickle_file(x, english_vocab)

In [ ]:
english_sentences = 'english_sentences.pkl'
save_pickle_file(corpus, english_sentences)

In [ ]:
hindi_sent = 'hindi_sentences.pkl'
save_pickle_file(hindi_sentences, hindi_sent)

In [ ]:
filename = 'english_vocab.pkl'
english_vocab = load_pickle_file(filename)

In [ ]:
len(english_vocab)

In [ ]:
english_vocab

In [ ]:
filename = 'english_sentences.pkl'
english_sentences = load_pickle_file(filename)

In [ ]:
english_sentences

In [ ]:
corpus_for_dictionary = ' '.join(english_sentences)

In [ ]:
corpus_for_dictionary

In [ ]:
corpus_for_dictionary1=[]
corpus_for_dictionary1.append(corpus_for_dictionary)

In [ ]:
corpus_for_dictionary1

In [ ]:
# lines = ['he is smile walk good person']

from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical


# size_of_vocabulary=5 #including one unknown word

tokenizer = Tokenizer(oov_token='UNK')

#creating dictionary with word as key and index as value
tokenizer.fit_on_texts(corpus_for_dictionary1)

#indexing of tokenizer starts from 1 not from 0 and 0 is used to pad so to differentiate, it is reserved.0
#setting unknown word token index
# tokenizer.word_index[tokenizer.oov_token] = num_words + 1 

print(tokenizer.word_index)

# example = array(tokenizer.texts_to_sequences(lines))

# vocabulary size
vocab_size = len(tokenizer.word_index)

print(vocab_size)

# encoded = to_categorical(example)

# print(encoded)

In [ ]:
print(vocab_size)

In [ ]:
eng_lines['English Sentence'][0]

In [ ]:
sen1 = []
sen1.append(eng_lines['English Sentence'][0])
type(sen1)

In [ ]:
from numpy import array
from keras.utils import to_categorical

In [ ]:
example = array(tokenizer.texts_to_sequences(sen1))

encoded = to_categorical(example)

print(encoded)

In [ ]:
sen1

In [ ]:
ex = tokenizer.texts_to_sequences(sen1)

In [ ]:
ex

In [ ]:
sen2 = ['tauji The is application nalsar']

In [ ]:
example = array(tokenizer.texts_to_sequences(sen2))

encoded = to_categorical(example,num_classes=vocab_size+1)

print(encoded[0])

In [ ]:
ex2 = tokenizer.texts_to_sequences(sen2)
print(ex2)

In [ ]:
encoded[0].shape

In [ ]:
filename = 'hindi_sentences.pkl'
hindi_sentences = load_pickle_file(filename)

In [ ]:
hindi_sentences[5]

In [ ]:
hindi_corpus_dictionary = ' '.join(hindi_sentences)

In [ ]:
corpus_for_dictionary2=[]
corpus_for_dictionary2.append(hindi_corpus_dictionary)

In [ ]:
# lines = ['he is smile walk good person']

from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical


size_of_vocabulary=90000 
#including one unknown word

tokenizer = Tokenizer(num_words=size_of_vocabulary, oov_token='UNK')

#creating dictionary with word as key and index as value
tokenizer.fit_on_texts(corpus_for_dictionary2)

#indexing of tokenizer starts from 1 not from 0 and 0 is used to pad so to differentiate, it is reserved.0
tokenizer.word_index = {e:i for e,i in tokenizer.word_index.items() if i <= size_of_vocabulary}

#setting unknown word token index
# tokenizer.word_index[tokenizer.oov_token] = num_words + 1 

print(tokenizer.word_index)

# example = array(tokenizer.texts_to_sequences(lines))

# vocabulary size
vocab_size = len(tokenizer.word_index)

print(vocab_size)

# encoded = to_categorical(example)

# print(encoded)

In [ ]:
print(vocab_size)

In [ ]:
sen1 = []
sen1.append(hin_lines['Hindi Sentence'][0])
type(sen1)

In [ ]:
example = array(tokenizer.texts_to_sequences(sen1))

encoded = to_categorical(example,num_classes=vocab_size+1)

print(encoded[0])

In [ ]:
ex2 = tokenizer.texts_to_sequences(sen1)
print(ex2)

In [ ]:
hin_lines['Hindi Sentence'][0]

In [ ]:
sen2 = []
sen2.append(hin_lines['Hindi Sentence'][1])
type(sen2)

In [ ]:
example = array(tokenizer.texts_to_sequences(sen2))

encoded = to_categorical(example,num_classes=vocab_size+1)

print(encoded)

In [ ]:
ex2 = tokenizer.texts_to_sequences(sen2)
print(ex2)

In [ ]:
hin_lines['Hindi Sentence'][1]